#### imports etc.

In [28]:
import pandas as pd
import folium
import json
import os.path
from pathlib import Path
resources = Path('./resources/')
from simple_salesforce import Salesforce

# Enter the relevant credentials for your Salesforce org.
sf = Salesforce(password='xxxxxxxxxxxxx', \
                username='xxxxxxxxxxxxx', \
                organizationId='xxxxxxxxxxxxx',\
                security_token='xxxxxxxxxxxxx')

In [2]:
# import uk postcodes csv file and look at shape of the data
postcodes_df = pd.read_csv(resources/'ukpostcodes.csv')

In [3]:
# Look at the shape of the postcodes data
postcodes_df.head()

,id,postcode,latitude,longitude
0,1,AB10 1XG,57.144165,-2.114848
1,2,AB10 6RN,57.137880,-2.121487
2,3,AB10 7JB,57.124274,-2.127190
3,4,AB11 5QN,57.142701,-2.093015
4,5,AB11 6UL,57.137547,-2.112696


In [15]:
# Query Salesforce for all UK Leads with Postcode values stored.
# We're also including the price range min/max fields to map the budget of leads across the country.
query= """
SELECT Id, PostalCode, Country, City, Price_Range_min__c, Price_Range_max__c 
FROM Lead
WHERE Country = 'United Kingdom' AND PostalCode != Null
"""
leads_df = pd.DataFrame(sf.query_all(query)['records'])
leads_df.drop('attributes',axis=1,inplace=True)
leads_df.head()

,City,Country,Id,PostalCode,Price_Range_max__c,Price_Range_min__c
0,London,United Kingdom,00Qb000000Fqx24EAB,SW8 3TH,500000.0,400001.0
1,Hersham,United Kingdom,00Qb000000Fqx1zEAB,KT12 5EA,1500000.0,1000001.0
2,London,United Kingdom,00Qb000000FrVppEAF,N4 3BP,500000.0,400001.0
3,London,United Kingdom,00Qb000000GUIGEEA5,SE13 5HT,400000.0,0.0
4,London,United Kingdom,00Qb000000Fqx0mEAB,SW10 0QF,400000.0,0.0


In [16]:
# Now we'll merge the Salesforce leads with the postcodes file to get the longitude/latitude values.
final_df = pd.merge(leads_df,postcodes_df,how='inner',left_on='PostalCode',right_on='postcode')
final_df.head()

,City,Country,Id,PostalCode,Price_Range_max__c,Price_Range_min__c,id,postcode,latitude,longitude
0,London,United Kingdom,00Qb000000Fqx24EAB,SW8 3TH,500000.0,400001.0,237039,SW8 3TH,51.469616,-0.147917
1,Hersham,United Kingdom,00Qb000000Fqx1zEAB,KT12 5EA,1500000.0,1000001.0,982955,KT12 5EA,51.359866,-0.424954
2,London,United Kingdom,00Qb000000FrVppEAF,N4 3BP,500000.0,400001.0,761309,N4 3BP,51.568071,-0.117744
3,London,United Kingdom,00Qb000000GUIGEEA5,SE13 5HT,400000.0,0.0,393916,SE13 5HT,51.458734,-0.004264
4,London,United Kingdom,00Qb000000Fqx0mEAB,SW10 0QF,400000.0,0.0,257307,SW10 0QF,51.479730,-0.179349


In [17]:
print leads_df.Id.count() - final_df.Id.count(), 'leads don\'t have a valid UK postcode.'

223 leads don't have a valid UK postcode.


### create some maps with folium

#### Start by creating a base map centred on UK coordinates

In [29]:
uk_start_coordinates = (53.99, -3.44)
map1 = folium.Map(location=uk_start_coordinates, zoom_start=6,tiles='Stamen Toner')
map1

The final dataframe is too large (>6k rows) to add markers for each data point (it can be done but is far too computationally expensive - look at the example jpeg in the repo to see how cluttered the markers appear). For demo purposes we'll add a random selection of 100 markers from the dataset.

In [19]:
for i in range (2000,2101,1):
    folium.Marker([final_df.latitude[i],final_df.longitude[i]], popup=final_df.Id[i],\
                  icon = folium.Icon(color ='red')).add_to(map1)
map1

Even within this subset of the data we can see that the leads are mostly coming from around London, which we would expect for a property development in London.

Since we can't add in every marker instead we'll focus on London and map number of leads within each London borough (using GEOJSON files taken from: https://github.com/utisz/compound-cities).

In [24]:
# Overlay the London boroughs
map3 = folium.Map(location=[51.515419, -0.141099], zoom_start=10)

for filename in os.listdir('./resources/inner_london_json/'):
    json_filepath = json.load(open('./resources/inner_london_json/'+filename))
    folium.GeoJson(json_filepath).add_to(map3)

for filename in os.listdir('./resources/outer_london_json/'):
    json_filepath = json.load(open('./resources/outer_london_json/'+filename))
    folium.GeoJson(json_filepath).add_to(map3)

In [25]:
map3